# Path pattern generation using an Evolutionary computing aproach

in this module we will explore on how we can generate a valid sequence based on genetic and evolutionary computing.

## Libraries

In [1]:
import random
import numpy as np
import cv2 as cv
import ppg_init as ppg
import seqeval

DEBUGMODE = False

initialized data stored in the following directory:
/d/APh504/Bachelor/Semester 10/1.project/my_project/offline_processing


## Fetch preprocessed data

##### ranked based selection
with linear rank probability
$$\frac{(2-s)}{\mu} + \frac{(2 i (s-1)}{\mu (\mu - 1)}$$

with exponential rank probability
$$ \frac{1 - e ^ {-i}}{c} $$

## Training path generating

### Training for separate digits

### Training for all digits and general model

In [23]:
# hello = np.array(['a', 'b', 'c'])
# world = np.array([1, 2])
# hello[world]

In [24]:
# for i in range(np.array([1, 2, 3, 4, 5]).shape[0]):
#     print(i)

In [25]:
# a= np.array([i for i in range(10)])
# b= np.cumsum(a)
# print(b)

In [26]:
# a[a>5]

In [27]:
# b[a>5]

In [28]:
# b[b>5]

In [29]:
# b[-1]

In [30]:
# b[[1,4,3]]

In [31]:
# list(b)

In [32]:
# def swap1(a):
#     a[0],a[1] = a[1],a[0]

# a = [0, 1]
# swap1(a)
# print(a)

In [33]:
# a[0:2]

In [34]:
# a = [1, 2, 4, 5, 6, 7]
# a[2:6][::-1]

In [35]:
# a = [(1,2)]
# (2, 1) in a

In [36]:
# a = [1, 2, 3, 4, 5]
# b = []
# b.append(a)
# b.append(a)
# b

In [37]:
# a= [1,2,3,4,5,6,7,8,9]
# b = a
# b[2:5] = [2222]
# print(a)

In [38]:
# a= [1,2,3,4,5,6,7,8,9]
# b = a[:]
# b[2:5] = [2222]
# print(a)
# print(b)

In [39]:
# partially_mapped_crossover([list(range(1,10)),[9,3,7,8,2,6,5,1,4]],get_parents_pair_nodup)

In [40]:
# np.array([-1]*8) + 1

In [41]:
# chrom_size = 10
# table = [[] for i in range(chrom_size)]
# print(table)
# table[1].append((2,False))
# table[1].append((3,True))
# print(table)

In [42]:
# a = [(x, True) for x in range(10)]
# a.remove((1, True))
# print(a)
# a.remove()

In [43]:
# a = [[],[]]
# a.empty()

In [44]:
# random.choice([])

In [45]:
# a = [1, 2, 4]
# b = np.cumsum(a)
# print(a)
# print(b)
# type(b)

In [46]:
# for i in range(50):
#     r =random.randint(0,10)
#     print(r)